Test bodo snowflake connectivity

In [8]:
%restart_ipy

Successfully restarted IPyParallel cluster. Please restart the kernels on your notebooks before using IPyParallel again.


In [1]:
%autopx

%autopx enabled


In [2]:
import bodo
print(bodo.__version__)

[stdout:0] 
2021.12.2
[stdout:1] 
2021.12.2
[stdout:2] 
2021.12.2
[stdout:3] 
2021.12.2
[stdout:4] 
2021.12.2
[stdout:5] 
2021.12.2
[stdout:6] 
2021.12.2
[stdout:7] 
2021.12.2


In [2]:
import snowflake.connector
#import bodo
import time
from sqlalchemy.dialects import registry
registry.register('snowflake', 'snowflake.sqlalchemy', 'dialect')

In [3]:
import os
with open("/shared/snowflake_testing/snowflake-secrets.txt","r") as f:
    for line in f:
        args=line.split("=")
        os.environ[args[0]]=args[1].strip()
username=os.environ["username"]
password=os.environ["password"]
account=os.environ["account"]
warehouse="BODO_VW"
database="BODO_DB"
role="BODO_R1"
#os.environ["BODO_LICENSE"]="FAEAABAnAADmBwAABAAAABIAAAAKfaiaykZF5GIBTEIUyHvVZRD+uXkV8eCAQeRxAG3kcXtLXM/NDPLM26E/iUxq+MNZ3xCAi2JnNK4kOQMOlBcWpwkJaRHzazPLY9lhpk453sI19kNi7w6Smuu3S+KGZK6sJBMO81Umi9VA9Zq5Y9CzQQgtJzxt+JKV6bbKJJryrRWFurrvbWAxwAfWFkdEJs+oHT23m/XhY4qVHBbHbebkeOis8RptIcxsQG7+j1frJ/dHb90VFNMDDMIa1aTlzxU99QLbSInRRwr5zZwR0RrHLrZcUX2WaSzmOaxbWxxiQCYFN7A8ut87p4A0aY2qXocUUwpbsUptovkuXUwzX+Ub"

In [ ]:
import bodo
import pandas as pd
@bodo.jit()
def test_case2a():
    df=pd.read_sql('select * from PUBLIC.TRIPS', f"snowflake://{username}:{password}@{account}/{database}/public?role={role}&warehouse={warehouse}",)
    df=df[df["tripduration"]<181]
    print(f"No. of records retrieved from trips table {len(df)}")
    return df
@bodo.jit()
def write_data(df):
    df=df.head(10)
    df.to_sql("PUBLIC.TRIPS2",f"snowflake://{username}:{password}@{account}/{database}/public?role={role}&warehouse={warehouse}",if_exists="append",index=False)
df=test_case2a()
password="BODO%40U1"
write_data(df)

In [6]:
import bodo
import pandas as pd
@bodo.jit()
def get_trips_data():
    df=pd.read_sql('select * from PUBLIC.TRIPS', f"snowflake://{username}:{password}@{account}/{database}/public?role={role}&warehouse={warehouse}",)
    df["start_year"] = pd.DatetimeIndex(df['starttime']).year
    df["birth_year"] = df["birth_year"].fillna(int(df["birth_year"].median()))
    df["age"]=df["start_year"]-df["birth_year"]
    df=df[df["age"]<90]
    print(f' records with age  > 90 = {len(df[df["age"]>90])}')
    print(f' records with age <= 90 ={len(df[df["age"]<=90])}')
    return df
password=os.environ["password"]
df=get_trips_data()

[stdout:0] 
 records with age  > 90 = 0
 records with age <= 90 =61438210


In [5]:
df


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,membership_type,usertype,birth_year,gender,start_year,age
0,848,2018-01-16 08:44:42,2018-01-16 08:58:51,3415,Prospect Pl & 6 Ave,40.679331,-73.975195,244,Willoughby Ave & Hall St,40.691960,-73.965369,31559,Annual Membership,Subscriber,1981,2,2018,37
1,500,2018-01-16 08:44:43,2018-01-16 08:53:04,385,E 55 St & 2 Ave,40.757973,-73.966033,173,Broadway & W 49 St,40.760683,-73.984527,30309,Annual Membership,Subscriber,1987,1,2018,31
2,2237,2018-01-16 08:44:43,2018-01-16 09:22:01,416,Cumberland St & Lafayette Ave,40.687534,-73.972652,455,1 Ave & E 44 St,40.750020,-73.969053,18866,Annual Membership,Subscriber,1990,1,2018,28
3,429,2018-01-16 08:44:45,2018-01-16 08:51:55,276,Duane St & Greenwich St,40.717488,-74.010455,360,William St & Pine St,40.707179,-74.008873,17049,Annual Membership,Subscriber,1984,1,2018,34
4,278,2018-01-16 08:44:45,2018-01-16 08:49:24,473,Rivington St & Chrystie St,40.721101,-73.991925,128,MacDougal St & Prince St,40.727103,-74.002971,15603,Annual Membership,Subscriber,1988,1,2018,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,908,2017-03-06 14:28:02,2017-03-06 14:43:11,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,3395,Henry St & W 9 St,40.676374,-74.003250,25102,NaN,Subscriber,1964,2,2017,53
12496,1061,2017-03-06 14:28:02,2017-03-06 14:45:43,456,E 53 St & Madison Ave,40.759711,-73.974023,507,E 25 St & 2 Ave,40.739126,-73.979738,19115,NaN,Subscriber,1958,1,2017,59
12497,260,2017-03-06 14:28:04,2017-03-06 14:32:25,3223,E 55 St & 3 Ave,40.758997,-73.968654,3142,1 Ave & E 62 St,40.761227,-73.960940,20124,NaN,Subscriber,1971,1,2017,46
12498,744,2017-03-06 14:28:05,2017-03-06 14:40:30,502,Henry St & Grand St,40.714215,-73.981346,285,Broadway & E 14 St,40.734546,-73.990741,17125,NaN,Subscriber,1947,1,2017,70


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,membership_type,usertype,birth_year,gender,start_year,age
12500,1584,2017-03-06 14:28:14,2017-03-06 14:54:38,514,12 Ave & W 40 St,40.760875,-74.002777,347,Greenwich St & W Houston St,40.728846,-74.008591,25734,NaN,Subscriber,1956,1,2017,61
12501,805,2017-03-06 14:28:16,2017-03-06 14:41:41,3260,Mercer St & Bleecker St,40.727064,-73.996621,527,E 33 St & 2 Ave,40.744023,-73.976056,26094,NaN,Subscriber,1984,1,2017,33
12502,348,2017-03-06 14:28:16,2017-03-06 14:34:04,3260,Mercer St & Bleecker St,40.727064,-73.996621,438,St Marks Pl & 1 Ave,40.727791,-73.985649,20142,NaN,Subscriber,1973,2,2017,44
12503,296,2017-03-06 14:28:18,2017-03-06 14:33:14,493,W 45 St & 6 Ave,40.756800,-73.982912,519,Pershing Square North,40.751873,-73.977706,16982,NaN,Subscriber,1972,1,2017,45
12504,1378,2017-03-06 14:28:20,2017-03-06 14:51:19,450,W 49 St & 8 Ave,40.762272,-73.987882,3137,5 Ave & E 73 St,40.772828,-73.966853,19344,NaN,Subscriber,1990,1,2017,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1114,2016-06-15 18:23:46,2016-06-15 18:42:21,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,343,Clinton Ave & Flushing Ave,40.697940,-73.969868,19199,NaN,Customer,1980,0,2016,36
24996,912,2016-06-15 18:23:47,2016-06-15 18:39:00,79,Franklin St & W Broadway,40.719116,-74.006667,161,LaGuardia Pl & W 3 St,40.729170,-73.998102,23511,NaN,Subscriber,1966,1,2016,50
24997,2476,2016-06-15 18:23:47,2016-06-15 19:05:03,336,Sullivan St & Washington Sq,40.730477,-73.999061,344,Monroe St & Bedford Ave,40.685144,-73.953809,22554,NaN,Subscriber,1984,1,2016,32
24998,1940,2016-06-15 18:23:48,2016-06-15 18:56:08,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,3240,NYCBS Depot BAL - DYR,0.000000,0.000000,18736,NaN,Customer,1980,0,2016,36


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,membership_type,usertype,birth_year,gender,start_year,age
25000,245,2016-06-15 18:23:50,2016-06-15 18:27:55,311,Norfolk St & Broome St,40.717227,-73.988021,502,Henry St & Grand St,40.714215,-73.981346,23112,NaN,Subscriber,1975,1,2016,41
25001,438,2016-06-15 18:23:51,2016-06-15 18:31:10,386,Centre St & Worth St,40.714948,-74.002345,263,Elizabeth St & Hester St,40.717290,-73.996375,25093,NaN,Subscriber,1985,1,2016,31
25002,575,2016-06-15 18:23:51,2016-06-15 18:33:27,518,E 39 St & 2 Ave,40.747804,-73.973442,477,W 41 St & 8 Ave,40.756405,-73.990026,14961,NaN,Subscriber,1988,1,2016,28
25003,1485,2016-06-15 18:23:51,2016-06-15 18:48:37,519,Pershing Square North,40.751873,-73.977706,320,Leonard St & Church St,40.717571,-74.005549,20061,NaN,Subscriber,1985,1,2016,31
25004,575,2016-06-15 18:23:51,2016-06-15 18:33:27,3119,Vernon Blvd & 50 Ave,40.742327,-73.954117,3112,Milton St & Franklin St,40.729060,-73.957790,19292,NaN,Subscriber,1988,1,2016,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37495,85,2014-11-08 19:53:49,2014-11-08 19:55:14,312,Allen St & E Houston St,40.722055,-73.989111,265,Stanton St & Chrystie St,40.722293,-73.991475,15453,NaN,Subscriber,1972,1,2014,42
37496,732,2014-11-08 19:53:51,2014-11-08 20:06:03,462,W 22 St & 10 Ave,40.746920,-74.004519,368,Carmine St & 6 Ave,40.730386,-74.002150,15151,NaN,Subscriber,1989,1,2014,25
37497,754,2014-11-08 19:53:52,2014-11-08 20:06:26,432,E 7 St & Avenue A,40.726218,-73.983799,540,Lexington Ave & E 26 St,40.741473,-73.983209,17844,NaN,Subscriber,1983,1,2014,31
37498,797,2014-11-08 19:53:55,2014-11-08 20:07:12,468,Broadway & W 55 St,40.765265,-73.981923,485,W 37 St & 5 Ave,40.750380,-73.983390,16013,NaN,Subscriber,1988,1,2014,26


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,membership_type,usertype,birth_year,gender,start_year,age
37500,1453,2014-11-08 19:53:59,2014-11-08 20:18:12,251,Mott St & Prince St,40.723180,-73.994800,327,Vesey Pl & River Terrace,40.715338,-74.016584,19951,NaN,Customer,1980,0,2014,34
37501,164,2014-11-08 19:54:00,2014-11-08 19:56:44,146,Hudson St & Reade St,40.716250,-74.009106,330,Reade St & Broadway,40.714505,-74.005628,17836,NaN,Subscriber,1986,1,2014,28
37502,818,2014-11-08 19:54:07,2014-11-08 20:07:45,458,11 Ave & W 27 St,40.751396,-74.005226,422,W 59 St & 10 Ave,40.770513,-73.988038,16443,NaN,Subscriber,1966,1,2014,48
37503,339,2014-11-08 19:54:08,2014-11-08 19:59:47,511,E 14 St & Avenue B,40.729387,-73.977724,483,E 12 St & 3 Ave,40.732233,-73.988900,16694,NaN,Subscriber,1991,1,2014,23
37504,222,2014-11-08 19:54:09,2014-11-08 19:57:51,475,E 16 St & Irving Pl,40.735243,-73.987586,168,W 18 St & 6 Ave,40.739713,-73.994564,18050,NaN,Subscriber,1987,1,2014,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,564,2015-10-18 12:36:58,2015-10-18 12:46:22,3126,44 Dr & Jackson Ave,40.747182,-73.943264,3124,46 Ave & 5 St,40.747310,-73.954510,24080,NaN,Subscriber,1976,1,2015,39
49996,546,2015-10-18 12:36:59,2015-10-18 12:46:06,248,Laight St & Hudson St,40.721854,-74.007718,264,Maiden Ln & Pearl St,40.707065,-74.007319,17734,NaN,Subscriber,1972,1,2015,43
49997,174,2015-10-18 12:37:02,2015-10-18 12:39:57,498,Broadway & W 32 St,40.748549,-73.988084,486,Broadway & W 29 St,40.746201,-73.988557,22559,NaN,Subscriber,1985,1,2015,30
49998,493,2015-10-18 12:37:03,2015-10-18 12:45:17,394,E 9 St & Avenue C,40.725213,-73.977688,502,Henry St & Grand St,40.714215,-73.981346,23975,NaN,Subscriber,1943,2,2015,72


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,membership_type,usertype,birth_year,gender,start_year,age
50000,738,2015-10-18 12:37:07,2015-10-18 12:49:25,259,South St & Whitehall St,40.701221,-74.012342,331,Pike St & Monroe St,40.711731,-73.991930,22746,NaN,Subscriber,1990,2,2015,25
50001,446,2015-10-18 12:37:07,2015-10-18 12:44:33,3170,W 84 St & Columbus Ave,40.785000,-73.972834,3175,W 70 St & Amsterdam Ave,40.777480,-73.982886,22226,NaN,Subscriber,1992,1,2015,23
50002,1014,2015-10-18 12:37:08,2015-10-18 12:54:02,520,W 52 St & 5 Ave,40.759923,-73.976485,161,LaGuardia Pl & W 3 St,40.729170,-73.998102,21332,NaN,Subscriber,1977,1,2015,38
50003,214,2015-10-18 12:37:09,2015-10-18 12:40:44,380,W 4 St & 7 Ave S,40.734011,-74.002939,116,W 17 St & 8 Ave,40.741776,-74.001497,17950,NaN,Subscriber,1977,1,2015,38
50004,560,2015-10-18 12:37:09,2015-10-18 12:46:29,489,10 Ave & W 28 St,40.750664,-74.001768,402,Broadway & E 22 St,40.740343,-73.989551,24309,NaN,Subscriber,1979,1,2015,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62495,5394,2013-12-22 07:50:45,2013-12-22 09:20:39,2021,W 45 St & 8 Ave,40.759291,-73.988597,447,8 Ave & W 52 St,40.763707,-73.985162,15027,NaN,Customer,1980,0,2013,33
62496,1696,2013-12-22 07:50:46,2013-12-22 08:19:02,474,5 Ave & E 29 St,40.745168,-73.986831,426,West St & Chambers St,40.717548,-74.013221,20277,NaN,Subscriber,1981,2,2013,32
62497,1095,2013-12-22 07:50:55,2013-12-22 08:09:10,82,St James Pl & Pearl St,40.711174,-74.000165,153,E 40 St & 5 Ave,40.752062,-73.981632,21035,NaN,Subscriber,1961,1,2013,52
62498,798,2013-12-22 07:50:58,2013-12-22 08:04:16,521,8 Ave & W 31 St,40.750450,-73.994811,281,Grand Army Plaza & Central Park S,40.764397,-73.973715,18559,NaN,Subscriber,1965,1,2013,48


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,membership_type,usertype,birth_year,gender,start_year,age
62500,801,2013-12-22 07:52:12,2013-12-22 08:05:33,434,9 Ave & W 18 St,40.743174,-74.003664,251,Mott St & Prince St,40.723180,-73.994800,19197,NaN,Subscriber,1956,1,2013,57
62501,253,2013-12-22 07:52:45,2013-12-22 07:56:58,406,Hicks St & Montague St,40.695128,-73.995951,157,Henry St & Atlantic Ave,40.690893,-73.996123,18178,NaN,Customer,1980,0,2013,33
62502,121,2013-12-22 07:53:10,2013-12-22 07:55:11,268,Howard St & Centre St,40.719105,-73.999733,268,Howard St & Centre St,40.719105,-73.999733,18944,NaN,Subscriber,1964,1,2013,49
62503,95,2013-12-22 07:53:10,2013-12-22 07:54:45,507,E 25 St & 2 Ave,40.739126,-73.979738,174,E 25 St & 1 Ave,40.738177,-73.977387,17444,NaN,Subscriber,1980,1,2013,33
62504,443,2013-12-22 07:53:19,2013-12-22 08:00:42,494,W 26 St & 8 Ave,40.747348,-73.997236,479,9 Ave & W 45 St,40.760193,-73.991255,17941,NaN,Subscriber,1983,1,2013,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74995,1212,2016-09-05 18:19:38,2016-09-05 18:39:51,507,E 25 St & 2 Ave,40.739126,-73.979738,3137,5 Ave & E 73 St,40.772828,-73.966853,15222,NaN,Subscriber,1962,1,2016,54
74996,1598,2016-09-05 18:19:38,2016-09-05 18:46:17,3223,E 55 St & 3 Ave,40.758997,-73.968654,3374,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955613,24911,NaN,Subscriber,1970,1,2016,46
74997,446,2016-09-05 18:19:39,2016-09-05 18:27:05,426,West St & Chambers St,40.717548,-74.013221,2008,Little West St & 1 Pl,40.705693,-74.016777,23773,NaN,Subscriber,1975,1,2016,41
74998,2497,2016-09-05 18:19:40,2016-09-05 19:01:17,387,Centre St & Chambers St,40.712733,-74.004607,387,Centre St & Chambers St,40.712733,-74.004607,20563,NaN,Customer,1980,0,2016,36


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,membership_type,usertype,birth_year,gender,start_year,age
75000,1614,2016-09-05 18:19:41,2016-09-05 18:46:36,3144,E 81 St & Park Ave,40.776777,-73.959010,3292,5 Ave & E 93 St,40.785785,-73.957481,16980,NaN,Customer,1980,0,2016,36
75001,1407,2016-09-05 18:19:41,2016-09-05 18:43:09,3173,Riverside Blvd & W 67 St,40.777507,-73.988886,525,W 34 St & 11 Ave,40.755942,-74.002116,16310,NaN,Customer,1980,0,2016,36
75002,701,2016-09-05 18:19:43,2016-09-05 18:31:25,72,W 52 St & 11 Ave,40.767272,-73.993929,3164,Columbus Ave & W 72 St,40.777057,-73.978985,25165,NaN,Subscriber,1957,1,2016,59
75003,1119,2016-09-05 18:19:43,2016-09-05 18:38:22,460,S 4 St & Wythe Ave,40.712859,-73.965903,3241,Monroe St & Tompkins Ave,40.686203,-73.944694,25622,NaN,Subscriber,1988,1,2016,28
75004,790,2016-09-05 18:19:43,2016-09-05 18:32:54,3144,E 81 St & Park Ave,40.776777,-73.959010,522,E 51 St & Lexington Ave,40.757148,-73.972078,16097,NaN,Customer,1980,0,2016,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87495,1236,2016-09-06 06:42:32,2016-09-06 07:03:09,119,Park Ave & St Edwards St,40.696089,-73.978034,412,Forsyth St & Canal St,40.715815,-73.994224,22455,NaN,Subscriber,1973,1,2016,43
87496,1398,2016-09-06 06:42:32,2016-09-06 07:05:50,487,E 20 St & FDR Drive,40.733143,-73.975739,320,Leonard St & Church St,40.717571,-74.005549,19517,NaN,Subscriber,1983,1,2016,33
87497,265,2016-09-06 06:42:36,2016-09-06 06:47:01,511,E 14 St & Avenue B,40.729387,-73.977724,297,E 15 St & 3 Ave,40.734232,-73.986923,15624,NaN,Subscriber,1996,1,2016,20
87498,294,2016-09-06 06:42:36,2016-09-06 06:47:30,3233,E 48 St & 5 Ave,40.757246,-73.978059,450,W 49 St & 8 Ave,40.762272,-73.987882,17997,NaN,Subscriber,1975,1,2016,41


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,membership_type,usertype,birth_year,gender,start_year,age
87500,222,2016-09-06 06:42:38,2016-09-06 06:46:21,376,John St & William St,40.708621,-74.007222,309,Murray St & West St,40.714979,-74.013012,20849,NaN,Subscriber,1976,1,2016,40
87501,1319,2016-09-06 06:42:38,2016-09-06 07:04:38,3230,Penn Station Valet,40.751284,-73.996924,330,Reade St & Broadway,40.714505,-74.005628,26918,NaN,Subscriber,1963,1,2016,53
87502,477,2016-09-06 06:42:40,2016-09-06 06:50:37,447,8 Ave & W 52 St,40.763707,-73.985162,359,E 47 St & Park Ave,40.755103,-73.974987,21341,NaN,Subscriber,1977,1,2016,39
87503,338,2016-09-06 06:42:40,2016-09-06 06:48:18,480,W 53 St & 10 Ave,40.766697,-73.990617,468,Broadway & W 55 St,40.765265,-73.981923,21055,NaN,Subscriber,1964,1,2016,52
87504,79,2016-09-06 06:42:41,2016-09-06 06:44:01,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3093,N 6 St & Bedford Ave,40.717452,-73.958509,16980,NaN,Subscriber,1974,1,2016,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,542,2014-05-11 11:52:03,2014-05-11 12:01:05,304,Broadway & Battery Pl,40.704633,-74.013617,315,South St & Gouverneur Ln,40.703554,-74.006702,16445,NaN,Customer,1980,0,2014,34
99996,196,2014-05-11 11:52:03,2014-05-11 11:55:19,453,W 22 St & 8 Ave,40.744751,-73.999154,494,W 26 St & 8 Ave,40.747348,-73.997236,21569,NaN,Subscriber,1983,1,2014,31
99997,707,2014-05-11 11:52:04,2014-05-11 12:03:51,462,W 22 St & 10 Ave,40.746920,-74.004519,480,W 53 St & 10 Ave,40.766697,-73.990617,19210,NaN,Subscriber,1974,1,2014,40
99998,2060,2014-05-11 11:52:07,2014-05-11 12:26:27,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,293,Lafayette St & E 8 St,40.730287,-73.990765,19524,NaN,Customer,1980,0,2014,34


In [ ]:
df

In [ ]:
import time
import snowflake.connector
def snowtest():
    t0 = time.time()
    print("start reading")
    cursor = snowflake.connector.connect(
        user=username,
        account=account,
        password=password,
        warehouse=warehouse,
        database=database,
        role=role
    ).cursor()
    sql = f'select * from PUBLIC.TRIPS limit 100000'
    cursor.execute(sql)
    df = cursor.fetch_pandas_all()
    #df = df.groupby("REF").sum()
    print(f"df read time connector: {round(time.time() - t0, 3)}")
    return df
import os
with open("/shared/snowflake_testing/snowflake-secrets.txt","r") as f:
    for line in f:
        args=line.split("=")
        os.environ[args[0]]=args[1].strip()
username=os.environ["username"]
password=os.environ["password"]
account=os.environ["account"]
warehouse="BODO_VW"
database="BODO_DB"
role="BODO_R1"
df1=snowtest()

In [ ]:
df1